# Case 3: No LDES to compare with Case 2

## Scheme 1 (c1s1b20 - No reserve requirements)

In [1]:
using CSV, DataFrames, JuMP, HiGHS, SCIP, Clp

storageinfo = CSV.read(joinpath("cleaned","storageinfo.csv"), DataFrame)
generatorinfo = CSV.read(joinpath("cleaned","generatorinfo.csv"), DataFrame)
vreinfo = CSV.read(joinpath("cleaned","vreinfo.csv"), DataFrame)
halfhourlydemand = CSV.read(joinpath("cleaned","halfhourlydemand.csv"), DataFrame)
halfhourlyvrecf = CSV.read(joinpath("cleaned","halfhourlyvrecf.csv"), DataFrame)
halfhourlydemand = halfhourlydemand[1:1440,:];

## Scheme 2 Fixed reserve requirements (2000MW)

In [2]:
function case3scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        GEN[G,T] >= 0
        R_GEN[G,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, TIC, GenInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, ReserveProvisionCost,
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * R_GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @objective(model, Min, EnergyProvisionCost + ReserveProvisionCost + UnservedEnergyCost + CurtailedVreCost)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] + R_GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])

    # VRE RESERVE REQUIREMENT CONSTRAINT
    @constraint(model, c24[t in T], 
        sum(R_GEN[g,t] for g in G) >= 2000)

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        GEN = value.(GEN).data,
        R_GEN = value.(R_GEN).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        ReserveProvisionCost = value(ReserveProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case3scheme2 (generic function with 1 method)

In [3]:
c3s2 = case3scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
11517 rows, 10134 cols, 36042 nonzeros
11516 rows, 10021 cols, 35928 nonzeros
Presolve : Reductions: rows 11516(-4321); columns 10021(-1501); elements 35928(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 2880(2.65584e+07) 0s
       2881     1.1309001594e+09 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2881
Objective value     :  1.1309001594e+09
HiGHS run time      :          0.03


(CAPG = [9554.923076923076, 51819.4], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], R_GEN = [0.0 0.0 … 0.0 0.0; 2000.0 2000.0 … 2000.0 2000.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 2.0e10, TIC = 2.0e10, EnergyProvisionCost = 9.16729056e8, ReserveProvisionCost = 1.944e8, UnservedEnergyCost = 0.0, CurtailedVreCost = 1.9771103400000032e7, OV = 1.1309001594000006e9)

## Scheme 3 Dynamic (Demand-based) Requirements (3% of Demand)

In [10]:
function case3scheme3(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        GEN[G,T] >= 0
        R_GEN[G,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, TIC, GenInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, ReserveProvisionCost,
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * R_GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @objective(model, Min, EnergyProvisionCost + ReserveProvisionCost + UnservedEnergyCost + CurtailedVreCost)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] + R_GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])

    # VRE RESERVE REQUIREMENT CONSTRAINT
    @constraint(model, c24[t in T], 
        sum(R_GEN[g,t] for g in G) >= 0.03 * halfhourlydemand[halfhourlydemand.hh.==t,:load][1])

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        GEN = value.(GEN).data,
        R_GEN = value.(R_GEN).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        ReserveProvisionCost = value(ReserveProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case3scheme3 (generic function with 1 method)

In [11]:
c3s3 = case3scheme3(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
11517 rows, 10134 cols, 36042 nonzeros
11516 rows, 10021 cols, 35928 nonzeros
Presolve : Reductions: rows 11516(-4321); columns 10021(-1501); elements 35928(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 2880(2.53784e+07) 0s
       2881     1.0512484894e+09 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2881
Objective value     :  1.0512484894e+09
HiGHS run time      :          0.03


(CAPG = [10245.966666666667, 51434.39], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], R_GEN = [0.0 0.0 … 0.0 0.0; 884.6999999999999 904.05 … 1020.06 992.61], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 2.0e10, TIC = 2.0e10, EnergyProvisionCost = 9.16729056e8, ReserveProvisionCost = 1.1474833004999977e8, UnservedEnergyCost = 0.0, CurtailedVreCost = 1.9771103400000032e7, OV = 1.0512484894499998e9)

In [21]:
## Scheme 4 Dynamic (VRE-based) Requirements (10% Wind 4% Solar)
function case3scheme4(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        GEN[G,T] >= 0
        R_GEN[G,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, TIC, GenInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, ReserveProvisionCost,
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * R_GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @objective(model, Min, EnergyProvisionCost + ReserveProvisionCost + UnservedEnergyCost + CurtailedVreCost)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] + R_GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])

    # VRE RESERVE REQUIREMENT CONSTRAINT
    @constraint(model, c24[t in T], 
        sum(R_GEN[g,t] for g in G) >= 0.04 * halfhourlyvrecf[halfhourlyvrecf.vre_id.==1,:cf][1] * vreinfo[vreinfo.id.==1,:installedMW][1] + 0.1 * (halfhourlyvrecf[halfhourlyvrecf.vre_id.==2,:cf][1] * vreinfo[vreinfo.id.==2,:installedMW][1] + halfhourlyvrecf[halfhourlyvrecf.vre_id.==3,:cf][1] * vreinfo[vreinfo.id.==3,:installedMW][1]))

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        GEN = value.(GEN).data,
        R_GEN = value.(R_GEN).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        ReserveProvisionCost = value(ReserveProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case3scheme4 (generic function with 1 method)

In [22]:
c3s4 = case3scheme4(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
11517 rows, 10134 cols, 36042 nonzeros
11516 rows, 10021 cols, 35928 nonzeros
Presolve : Reductions: rows 11516(-4321); columns 10021(-1501); elements 35928(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 2880(3.04756e+07) 0s
       2881     1.3953113754e+09 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2881
Objective value     :  1.3953113754e+09
HiGHS run time      :          0.03


(CAPG = [4672.369230769231, 54539.68], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], R_GEN = [0.0 0.0 … 0.0 0.0; 4720.280000000001 4720.280000000001 … 4720.280000000001 4720.280000000001], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 2.0e10, TIC = 2.0e10, EnergyProvisionCost = 9.16729056e8, ReserveProvisionCost = 4.588112159999893e8, UnservedEnergyCost = 0.0, CurtailedVreCost = 1.9771103400000032e7, OV = 1.3953113754000864e9)

In [29]:
## Scheme 5 ALL
function case3scheme5(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        GEN[G,T] >= 0
        R_GEN[G,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, TIC, GenInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, ReserveProvisionCost,
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * R_GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @objective(model, Min, EnergyProvisionCost + ReserveProvisionCost + UnservedEnergyCost + CurtailedVreCost)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] + R_GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])

    # VRE RESERVE REQUIREMENT CONSTRAINT
    @constraint(model, c24[t in T], 
        sum(R_GEN[g,t] for g in G) >= 2000 + 0.03 * halfhourlydemand[halfhourlydemand.hh.==t,:load][1] + 0.04 * halfhourlyvrecf[halfhourlyvrecf.vre_id.==1,:cf][1] * vreinfo[vreinfo.id.==1,:installedMW][1] + 0.1 * (halfhourlyvrecf[halfhourlyvrecf.vre_id.==2,:cf][1] * vreinfo[vreinfo.id.==2,:installedMW][1] + halfhourlyvrecf[halfhourlyvrecf.vre_id.==3,:cf][1] * vreinfo[vreinfo.id.==3,:installedMW][1]))

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        GEN = value.(GEN).data,
        R_GEN = value.(R_GEN).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        ReserveProvisionCost = value(ReserveProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case3scheme5 (generic function with 1 method)

In [30]:
c3s5 = case3scheme5(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
11517 rows, 10134 cols, 36042 nonzeros
11516 rows, 10021 cols, 35928 nonzeros
Presolve : Reductions: rows 11516(-4321); columns 10021(-1501); elements 35928(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 2880(3.50556e+07) 0s
       2888     1.7047172012e+09 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2888
Objective value     :  1.7047172012e+09
HiGHS run time      :          0.03


(CAPG = [2284.738709677411, 55869.93129032258], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], R_GEN = [0.0 0.0 … 0.0 0.0; 7604.9800000000005 7624.330000000001 … 7740.34 7712.890000000001], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 2.0e10, TIC = 2.0e10, EnergyProvisionCost = 9.169180096064516e8, ReserveProvisionCost = 7.680280882112911e8, UnservedEnergyCost = 0.0, CurtailedVreCost = 1.9771103400000032e7, OV = 1.7047172012177446e9)